<a href="https://colab.research.google.com/github/GGlivePh/PLF/blob/main/DolSensors.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!git clone https://github.com/GGlivePh/QG.git

In [ ]:
import os
os.chdir('QG')

In [ ]:
# !pip install azure-servicebus

In [ ]:
# API Credentials
API_BASE_URL = "https://iot.dol-sensors.com/api"
EMAIL = "thami@qgg.au.dk"  # Enclose in quotes to treat as string
PASSWORD = "Dolras@data321"

import requests
import json
import pandas as pd
from azure.servicebus import ServiceBusClient

# Step 1: Authenticate and Get Access Token
def get_access_token():
    auth_url = f"{API_BASE_URL}/auth/login"
    payload = {"email": EMAIL, "password": PASSWORD}

    response = requests.post(auth_url, json=payload)

    if response.status_code == 200:
        token = response.json().get("accessToken")
        print("Authentication successful!")
        return token
    else:
        print("Failed to authenticate:", response.status_code, response.text)
        return None

# Step 2: Claim Device (Gateway)
def claim_device(token, gateway_id, claim_code, device_type, customer_id, name):
    url = f"{API_BASE_URL}/devices/claim"
    headers = {"Authorization": f"Bearer {token}"}
    payload = {
        "gatewayId": gateway_id,
        "claimCode": claim_code,
        "deviceType": device_type,
        "customerId": customer_id,
        "name": name
    }
    response = requests.post(url, headers=headers, json=payload)
    if response.status_code == 200:
        print(" Gateway claimed successfully!")
    else:
        print(" Failed to claim gateway:")
        print("Status Code:", response.status_code)
        print("Response:", response.text)

# Step 3: Add LoRa Sensor
def add_lora_sensor(token, gateway_id, dev_eui, interval, sensor_type, name):
    url = f"{API_BASE_URL}/devices/{gateway_id}/sensors"
    headers = {"Authorization": f"Bearer {token}"}
    payload = {
        "devEui": dev_eui,
        "interval": interval,
        "sensorType": sensor_type,
        "name": name
    }
    response = requests.post(url, headers=headers, json=payload)
    if response.status_code == 200:
        print(" LoRa sensor added successfully!")
    else:
        print(" Failed to add LoRa sensor:")
        print("Status Code:", response.status_code)
        print("Response:", response.text)

# Step 4: Retrieve Queue Details
def get_queue_details(token):
    queue_url = f"{API_BASE_URL}/management/queue"
    headers = {"Authorization": f"Bearer {token}"}

    response = requests.get(queue_url, headers=headers)

    if response.status_code == 200:
        queues = response.json()
        print(" Queue details retrieved!")
        return queues.get("dataQueueConnection"), queues.get("dataQueueName")
    else:
        print(" Failed to get queue details:")
        print("Status Code:", response.status_code)
        print("Response:", response.text)
        return None, None

# Step 5: Fetch Data from the Queue
def fetch_sensor_data(queue_connection, queue_name):
    servicebus_client = ServiceBusClient.from_connection_string(queue_connection)

    with servicebus_client:
        receiver = servicebus_client.get_queue_receiver(queue_name, max_wait_time=5)

        print(" Listening to queue...")
        messages = []

        for msg in receiver:
            data = json.loads(str(msg))
            messages.append(data)
            receiver.complete_message(msg)  # Mark message as processed

        return messages

# Step 6: Save Data to CSV/JSON
def save_data(data, filename="sensor_data.csv"):
    df = pd.DataFrame(data)
    df.to_csv(filename, index=False)
    print(f" Data saved to {filename}")

# Main Execution
if __name__ == "__main__":
    token = get_access_token()

    if token:
        # Try claiming device and registering sensor first
        claim_device(token, "000ecd031af1", "nMNLPSNT", "IDOL64", "QGG-AU", "Test device")
        add_lora_sensor(token, "000ecd031af1", "f2b3d57ed8522302", "60", "IDOL139", "Sensor 1: Temperature, Humidity and CO2")

        # Then get queue details and fetch data
        queue_connection, queue_name = get_queue_details(token)

        if queue_connection and queue_name:
            sensor_data = fetch_sensor_data(queue_connection, queue_name)

            if sensor_data:
                save_data(sensor_data)
                print(" Data retrieval completed successfully!")
            else:
                print("No data received from the queue.")



# API Credentials
API_BASE_URL = "https://iot.dol-sensors.com/api"
EMAIL = "thami@qgg.au.dk"  # Enclose in quotes to treat as string
PASSWORD = "Dolras@data321"

import requests
# API Credentials
API_BASE_URL = "https://iot.dol-sensors.com/api"
EMAIL = "thami@qgg.au.dk"  # Enclose in quotes to treat as string
PASSWORD = "Dolras@data321"

import requests
import json
import pandas as pd
from azure.servicebus import ServiceBusClient

# Step 1: Authenticate and Get Access Token
def get_access_token():
    auth_url = f"{API_BASE_URL}/auth/login"
    payload = {"email": EMAIL, "password": PASSWORD}

    response = requests.post(auth_url, json=payload)

    if response.status_code == 200:
        token = response.json().get("accessToken")
        print("Authentication successful!")
        return token
    else:
        print("Failed to authenticate:", response.status_code, response.text)
        return None

# Step 2: Claim Device (Gateway)
def claim_device(token, gateway_id, claim_code, device_type, customer_id, name):
    url = f"{API_BASE_URL}/devices/claim"
    headers = {"Authorization": f"Bearer {token}"}
    payload = {
        "gatewayId": gateway_id,
        "claimCode": claim_code,
        "deviceType": device_type,
        "customerId": customer_id,
        "name": name
    }
    response = requests.post(url, headers=headers, json=payload)
    if response.status_code == 200:
        print(" Gateway claimed successfully!")
    else:
        print(" Failed to claim gateway:")
        print("Status Code:", response.status_code)
        print("Response:", response.text)

# Step 3: Add LoRa Sensor
def add_lora_sensor(token, gateway_id, dev_eui, interval, sensor_type, name):
    url = f"{API_BASE_URL}/devices/{gateway_id}/sensors"
    headers = {"Authorization": f"Bearer {token}"}
    payload = {
        "devEui": dev_eui,
        "interval": interval,
        "sensorType": sensor_type,
        "name": name
    }
    response = requests.post(url, headers=headers, json=payload)
    if response.status_code == 200:
        print(" LoRa sensor added successfully!")
    else:
        print(" Failed to add LoRa sensor:")
        print("Status Code:", response.status_code)
        print("Response:", response.text)

# Step 4: Retrieve Queue Details
def get_queue_details(token):
    queue_url = f"{API_BASE_URL}/management/queue"
    headers = {"Authorization": f"Bearer {token}"}

    response = requests.get(queue_url, headers=headers)

    if response.status_code == 200:
        queues = response.json()
        print(" Queue details retrieved!")
        return queues.get("dataQueueConnection"), queues.get("dataQueueName")
    else:
        print(" Failed to get queue details:")
        print("Status Code:", response.status_code)
        print("Response:", response.text)
        return None, None

# Step 5: Fetch Data from the Queue
def fetch_sensor_data(queue_connection, queue_name):
    servicebus_client = ServiceBusClient.from_connection_string(queue_connection)

    with servicebus_client:
        receiver = servicebus_client.get_queue_receiver(queue_name, max_wait_time=5)

        print(" Listening to queue...")
        messages = []

        for msg in receiver:
            data = json.loads(str(msg))
            messages.append(data)
            receiver.complete_message(msg)  # Mark message as processed

        return messages

# Step 6: Save Data to CSV/JSON
def save_data(data, filename="sensor_data.csv"):
    df = pd.DataFrame(data)
    df.to_csv(filename, index=False)
    print(f" Data saved to {filename}")

# Main Execution
if __name__ == "__main__":
    token = get_access_token()

    if token:
        # Try claiming device and registering sensor first
        claim_device(token, "000ecd031af1", "nMNLPSNT", "IDOL64", "QGG-AU", "Test device")
        add_lora_sensor(token, "000ecd031af1", "f2b3d57ed8522302", "60", "IDOL139", "Sensor 1: Temperature, Humidity and CO2")

        # Then get queue details and fetch data
        queue_connection, queue_name = get_queue_details(token)

        if queue_connection and queue_name:
            sensor_data = fetch_sensor_data(queue_connection, queue_name)

            if sensor_data:
                save_data(sensor_data)
                print(" Data retrieval completed successfully!")
            else:
                print("No data received from the queue.")
# Step 1: Authenticate and Get Access Token
def get_access_token():
    auth_url = f"{API_BASE_URL}/auth/login"
    payload = {"email": EMAIL, "password": PASSWORD}

    response = requests.post(auth_url, json=payload)

    if response.status_code == 200:
        token = response.json().get("accessToken")
        print("Authentication successful!")
        return token
    else:
        print("Failed to authenticate:", response.status_code, response.text)
        return None

# Step 2: Claim Device (Gateway)
def claim_device(token, gateway_id, claim_code, device_type, customer_id, name):
    url = f"{API_BASE_URL}/devices/claim"
    headers = {"Authorization": f"Bearer {token}"}
    payload = {
        "gatewayId": gateway_id,
        "claimCode": claim_code,
        "deviceType": device_type,
        "customerId": customer_id,
        "name": name
    }
    response = requests.post(url, headers=headers, json=payload)
    if response.status_code == 200:
        print(" Gateway claimed successfully!")
    else:
        print(" Failed to claim gateway:")
        print("Status Code:", response.status_code)
        print("Response:", response.text)

# Step 3: Add LoRa Sensor
def add_lora_sensor(token, gateway_id, dev_eui, interval, sensor_type, name):
    url = f"{API_BASE_URL}/devices/{gateway_id}/sensors"
    headers = {"Authorization": f"Bearer {token}"}
    payload = {
        "devEui": dev_eui,
        "interval": interval,
        "sensorType": sensor_type,
        "name": name
    }
    response = requests.post(url, headers=headers, json=payload)
    if response.status_code == 200:
        print(" LoRa sensor added successfully!")
    else:
        print(" Failed to add LoRa sensor:")
        print("Status Code:", response.status_code)
        print("Response:", response.text)

# Step 4: Retrieve Queue Details
def get_queue_details(token):
    queue_url = f"{API_BASE_URL}/management/queue"
    headers = {"Authorization": f"Bearer {token}"}

    response = requests.get(queue_url, headers=headers)

    if response.status_code == 200:
        queues = response.json()
        print(" Queue details retrieved!")
        return queues.get("dataQueueConnection"), queues.get("dataQueueName")
    else:
        print(" Failed to get queue details:")
        print("Status Code:", response.status_code)
        print("Response:", response.text)
        return None, None

# Step 5: Fetch Data from the Queue
def fetch_sensor_data(queue_connection, queue_name):
    servicebus_client = ServiceBusClient.from_connection_string(queue_connection)

    with servicebus_client:
        receiver = servicebus_client.get_queue_receiver(queue_name, max_wait_time=5)

        print(" Listening to queue...")
        messages = []

        for msg in receiver:
            data = json.loads(str(msg))
            messages.append(data)
            receiver.complete_message(msg)  # Mark message as processed

        return messages

# Step 6: Save Data to CSV/JSON
def save_data(data, filename="sensor_data.csv"):
    df = pd.DataFrame(data)
    df.to_csv(filename, index=False)
    print(f" Data saved to {filename}")

# Main Execution
if __name__ == "__main__":
    token = get_access_token()

    if token:
        # Try claiming device and registering sensor first
        claim_device(token, "000ecd031af1", "nMNLPSNT", "IDOL64", "QGG-AU", "Test device")
        add_lora_sensor(token, "000ecd031af1", "f2b3d57ed8522302", "60", "IDOL139", "Sensor 1: Temperature, Humidity and CO2")

        # Then get queue details and fetch data
        queue_connection, queue_name = get_queue_details(token)

        if queue_connection and queue_name:
            sensor_data = fetch_sensor_data(queue_connection, queue_name)

            if sensor_data:
                save_data(sensor_data)
                print(" Data retrieval completed successfully!")
            else:
                print("No data received from the queue.")
